# Loading needed libraries

In [46]:
#!pip install pandas matplotlib seaborn plotnine statsmodels scikit-learn scikit-posthocs dtreeviz
#!pip install pandas

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import numpy as np
from scipy.stats import shapiro
from scipy.stats import levene
from scipy.stats import kruskal
import scikit_posthocs as sp
from sklearn.tree import DecisionTreeClassifier
import graphviz
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import plot_tree
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score

#plt.style.use('seaborn-darkgrid')
#sns.set_context('notebook')

# Preparing Tables

In [7]:
# Reading CSV label files
label_table = pd.read_csv("label_dataset.csv")
#label_table.head(10)

commits_with_git_url = pd.read_csv("commits_with_git_url.csv")

# Joining dataframes
commits_label = pd.merge(commits_with_git_url, label_table, on="repo_id", how="inner")

#commits_label.head(10)
# Selecting specific columns
commitSelectedVar = commits_label[['repo_id', 'commits', 'label_name']]

commiters_number = pd.read_csv("committer_count_by_repo.csv", usecols=lambda column: column != 'Unnamed: 0')
#print(commiters_number.dtypes)

# Selecting specific columns
commitersSelectedVar = commiters_number[["repo_id", "committer_count"]]

commiters_18_time = pd.read_csv("committer_time.csv", usecols=lambda column: column != 'Unnamed: 0')
#print(commiters_18_time.dtypes)

# Selecting specific columns
commitersTimeSelectedVar = commiters_18_time[["repo_id", "committer_count"]]

# Rename the column in commitersByTimeSelectedVarS
commitersTimeSelectedVar = commitersTimeSelectedVar.rename(columns={'committer_count': 'committer_time'})

# Combine the datasets using inner join on 'repo_id'
combined_dataset = pd.merge(commitSelectedVar, commitersSelectedVar, on='repo_id', how='inner')
combined_dataset = pd.merge(combined_dataset, commitersTimeSelectedVar, on='repo_id', how='inner')

# Print the combined dataset's structure
print(combined_dataset.info())

# Write the combined dataset to a CSV file
combined_dataset.to_csv('combined_dataset.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo_id          160 non-null    int64 
 1   commits          160 non-null    int64 
 2   label_name       160 non-null    object
 3   committer_count  160 non-null    int64 
 4   committer_time   160 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 6.4+ KB
None


In [11]:
# Read the combined dataset
combined_dataset = pd.read_csv("combined_dataset.csv")
combined_dataset.head(10)

,repo_id,commits,label_name,committer_count,committer_time
0,165470,177000,grads,1121,324
1,165414,67209,grads,915,149
2,191242,3204,sandbox,32,15
3,191227,23171,sandbox,116,49
4,191142,31586,grads,1257,591
5,191281,14413,sandbox,79,58
6,191228,21940,sandbox,40,19
7,191236,46980,sandbox,75,34
8,191154,205155,incubating,1444,749
9,191194,12492,sandbox,98,6


# Running multinomial logistic regression

In [22]:
# Separating features and target variable
X = combined_dataset[['commits', 'committer_count', 'committer_time']]
y = combined_dataset['label_name']

# Performing Multinomial Logistic Regression
multinom_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
multinom_model.fit(X, y)

# Making predictions on the original dataset for demonstration purposes
predictions = multinom_model.predict(X)

# Adding predictions to the dataframe
combined_dataset['multinom_predicted_group'] = predictions

# Creating a confusion matrix
conf_matrix = confusion_matrix(y, predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Calculating and printing accuracy
accuracy = accuracy_score(y, predictions)
print(f"Classification accuracy: {accuracy}")

Confusion Matrix:
[[16  3  4]
 [22  1  9]
 [22 15 68]]
Classification accuracy: 0.53125


# Running linear discriminant analysis(LDA) model

In [23]:
# Performing LDA
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X, y)

# Making predictions on the original dataset for demonstration
predictions = lda_model.predict(X)

# Adding predictions to the dataframe
combined_dataset['lda_predicted_group'] = predictions

# Confusion Matrix
conf_matrix = confusion_matrix(y, predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Calculate accuracy
accuracy = accuracy_score(y, predictions)
print(f"Classification accuracy: {accuracy}")

Confusion Matrix:
[[  8   1  14]
 [  4   0  28]
 [  1   1 103]]
Classification accuracy: 0.69375


# Running Quadratic Discriminant Analysis(QDA) model

In [24]:
# Performing QDA
qda_model = QuadraticDiscriminantAnalysis()
qda_model.fit(X, y)

# Making predictions on the original dataset for demonstration purposes
predictions = qda_model.predict(X)

# Adding predictions to the dataframe
combined_dataset['qda_predicted_group'] = predictions

# Creating a confusion matrix
conf_matrix = confusion_matrix(y, predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Calculating and printing accuracy
accuracy = accuracy_score(y, predictions)
print(f"Classification accuracy: {accuracy}")

Confusion Matrix:
[[ 7 11  5]
 [ 3  7 22]
 [ 2  4 99]]
Classification accuracy: 0.70625


# Running decision tree model

In [26]:
# Fitting the decision tree model
tree_model = DecisionTreeClassifier()
tree_model.fit(X, y)

# Making predictions
predictions = tree_model.predict(X)

# Creating a confusion matrix
conf_matrix = confusion_matrix(y, predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Calculating accuracy
accuracy = accuracy_score(y, predictions)
print(f"Classification accuracy: {accuracy}")

Confusion Matrix:
[[ 23   0   0]
 [  0  32   0]
 [  0   0 105]]
Classification accuracy: 1.0


## Interpreting the decision tree result

In [31]:
# Plotting the tree
plt.figure(figsize=(200,100))

plot_tree(tree_model, filled=True, rounded=True, class_names=tree_model.classes_, feature_names=['commits', 'committer_count', 'committer_time'])
plt.title("Decision Tree for Group Classification")

# Save the figure
plt.savefig('decision_tree.png', format='png', bbox_inches='tight')

plt.show()

## How about the testing accuracy

In [56]:
# Split the data into training and test sets while preserving class distribution
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Initialize a StratifiedKFold object
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Initialize the decision tree classifier
tree_model = DecisionTreeClassifier(random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(tree_model, X_train, y_train, cv=skf, scoring='accuracy')

# Fit the model on the entire training data
tree_model.fit(X_train, y_train)

# Cross-validation scores
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean CV accuracy: {cv_scores.mean()}')

# Making predictions on the test set
predictions = tree_model.predict(X_test)

# Create a confusion matrix and calculate accuracy on the test set
conf_matrix = confusion_matrix(y_test, predictions)
accuracy = accuracy_score(y_test, predictions)

print("Confusion Matrix:")
print(conf_matrix)

print(f"Test Classification accuracy: {accuracy}")

# Additional detailed classification report
print("Classification Report:")
print(classification_report(y_test, predictions))

Cross-validation scores: [0.69230769 0.69230769 0.53846154 0.76923077 0.69230769 0.46153846
 0.76923077 0.76923077 0.58333333 0.58333333]
Mean CV accuracy: 0.6551282051282051
Confusion Matrix:
[[ 2  0  3]
 [ 0  4  2]
 [ 1  3 17]]
Test Classification accuracy: 0.71875
Classification Report:
              precision    recall  f1-score   support

       grads       0.67      0.40      0.50         5
  incubating       0.57      0.67      0.62         6
     sandbox       0.77      0.81      0.79        21

    accuracy                           0.72        32
   macro avg       0.67      0.63      0.64        32
weighted avg       0.72      0.72      0.71        32



In [54]:
# Retrive feature importances
feature_importances = tree_model.feature_importances_

feature_names = ['commits', 'committer_count', 'committer_time']
print("Feature importances:")
for name, importance in zip(feature_names, feature_importances):
    print(f"{name}: {importance:.4f}")

Feature importances:
commits: 0.2199
committer_count: 0.5726
committer_time: 0.2075


In [53]:
# Print the sorted importance rate
feature_importances_series = pd.Series(feature_importances, index=feature_names)
print(feature_importances_series.sort_values(ascending=False))

committer_count    0.572645
commits            0.219857
committer_time     0.207499
dtype: float64
